In [16]:
import time

import requests
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [17]:
fake = faker.Faker(locale='en')

In [18]:
headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'referer': 'http://careers.google.com/jobs/results/'
}

In [19]:
API_URL = "http://careers.google.com/api/v3/search/"
VACANCY_URL = 'http://careers.google.com/jobs/results/'

In [20]:
search_params = {
    'page': 1,
    # 'company': 'Google', # One of Alphabet holding's company
    'q': '',
    'sort_by': 'date',
    'page_size': 100 # must be 20, 50, 75, 100
}

In [21]:
result = requests.get(API_URL, params=search_params, headers=headers)
result

<Response [200]>

In [22]:
total_count = result.json()['count']
max_pages = total_count // 100
print(total_count, max_pages)

4026 40


In [23]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace("&nbsp;", "").replace("&amp;", "&").replace('\n', '\n ')).strip()
title_to_url = lambda x: "-".join(re.sub('\s+', ' ', re.sub(r'[^a-zA-Z\s]', '', x.lower())).split(' '))

In [24]:
search_results = []
null_counter = 0
for page in tqdm(range(max_pages)):
    search_params['page'] = page

    result = requests.get(API_URL, params=search_params, headers=headers).json()
    found_jobs = result['jobs']

    if len(found_jobs) == 0:
        null_counter += 1
        if null_counter == 3:
            break
    else:
        null_counter = 0

    for job_json in found_jobs:
        internal_id = job_json['id'].split('/')[-1]
        result_dict = {
            'title': job_json['title'],
            'internal_id': internal_id,
            'url': VACANCY_URL + internal_id + '-' + title_to_url(job_json['title']),
            'description': clear_string(job_json['description']),
            'responsibilities': clear_string(job_json['responsibilities']),
            'qualifications': clear_string(job_json['qualifications']),
            'company': 'Google',
            # 'country_code': job_json['country_code'],
            'publish_date': parser.parse(job_json['publish_date'])
        }
        search_results.append(result_dict)

    time.sleep(0.2) # sleep for 200 ms

100%|██████████| 40/40 [01:19<00:00,  1.98s/it]


In [30]:
snapshot = pd.DataFrame(search_results)
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
2489,"Enterprise Field Sales Representative, Financi...",92861367100285638,http://careers.google.com/jobs/results/9286136...,The Google Cloud Platform team helps customers...,Build and grow executive relationships with en...,Minimum qualifications: \n Bachelor's degree i...,Google,2022-05-11 02:31:50.717000+00:00
2877,"Electrical Technician, Data Center Facilities ...",119563303065133766,http://careers.google.com/jobs/results/1195633...,The Data Center team designs and operates some...,"Inspect, maintain, and repair various data cen...","Minimum qualifications: \n Associate's degree,...",Google,2022-04-21 13:02:09.457000+00:00
3105,"Software Engineer, University Graduate, 2022",124750385131725510,http://careers.google.com/jobs/results/1247503...,Google's software engineers develop the next-g...,"Build our platforms, systems, and networking i...",Minimum qualifications: \n Bachelor's degree i...,Google,2022-04-07 18:31:51.054000+00:00
3324,"Customer Engineer, SAP Specialist",90661397542642374,http://careers.google.com/jobs/results/9066139...,When leading companies choose Google Cloud it'...,Work with the team to identify and qualify bus...,Minimum qualifications: \n Bachelor's degree i...,Google,2022-03-22 15:31:59.026000+00:00
1384,"Workplace Services Manager, Occupancy Strategist",104181186957320902,http://careers.google.com/jobs/results/1041811...,The Real Estate and Workplace Services (REWS) ...,"Plan capacity and space, move coordination, an...",Minimum qualifications: \n Bachelors degree or...,Google,2022-06-16 16:31:45.805000+00:00


In [31]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype                  
---  ------            --------------  -----                  
 0   title             4000 non-null   object                 
 1   internal_id       4000 non-null   object                 
 2   url               4000 non-null   object                 
 3   description       4000 non-null   object                 
 4   responsibilities  4000 non-null   object                 
 5   qualifications    4000 non-null   object                 
 6   company           4000 non-null   object                 
 7   publish_date      4000 non-null   datetime64[ns, tzutc()]
dtypes: datetime64[ns, tzutc()](1), object(7)
memory usage: 250.1+ KB


In [32]:
snapshot['publish_date'] = snapshot['publish_date'].apply(lambda x: x.replace(tzinfo=None))
snapshot['publish_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4000 entries, 0 to 3999
Series name: publish_date
Non-Null Count  Dtype         
--------------  -----         
4000 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 31.4 KB


In [33]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [34]:
snapshot.to_csv(f'../data/google/{current_date}.csv')
snapshot.to_excel(f'../data/google/{current_date}.xlsx', engine='xlsxwriter')